In [1]:
#!/usr/bin/env python3
import pandas as pd
import lz4.frame
import gzip
import io
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from glob import glob
from plumbum.cmd import rm
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os.path
import pickle
import datetime
import re

Using TensorFlow backend.


In [2]:
def plotline(data):
    plt.figure()
    plt.plot(data)
    plt.legend()
    plt.show()

def event_count(time_series, data_name):
    time_series = time_series[['Fill Price (USD)']].values
    upevents = 0
    downevents = 0
    sameprice = 0
    prev_obv = time_series[0]
    for obv in time_series[1:]:
        if obv > prev_obv:
            upevents += 1
        elif obv < prev_obv:
            downevents += 1
        elif obv == prev_obv:
            sameprice += 1
        prev_obv = obv
    print('=== Event counts on %s ===' % data_name)
    print('upevents')
    print(upevents)
    print('downevents')
    print(downevents)
    print('sameprice')
    print(sameprice)
    print()

def mse(time_series, data_name):
    time_series = time_series[['Fill Price (USD)']].values
    total_squared_error = 0
    total_absolute_error = 0
    prev_obv = time_series[0]
    for obv in time_series[1:]:
        total_squared_error += (obv - prev_obv)**2
        total_absolute_error += abs(obv - prev_obv)
        prev_obv = obv
    num_predictions = len(time_series) - 1
    mean_squared_error = total_squared_error / num_predictions
    mean_absolute_error = total_absolute_error / num_predictions
    root_mean_squared_error = np.sqrt(mean_squared_error)
    print('=== baseline on %s ===' % data_name)
    print('total squared error')
    print(total_squared_error)
    print('total absolute error')
    print(total_absolute_error)
    print('mean squared error')
    print(mean_squared_error)
    print('mean absolute error')
    print(mean_absolute_error) 
    print('root mean squared error')
    print(root_mean_squared_error) 
    print()

In [6]:
def show_summary_statistics():
    #event_count(small_set, 'small')
    train_set = df.iloc[0:num_samples_training]
    dev_set = df.iloc[num_samples_training:num_samples_training+num_samples_dev]
    test_set = df.iloc[num_samples_training+num_samples_dev:]
    event_count(train_set, 'train')
    event_count(dev_set, 'dev')
    event_count(test_set, 'test')
    mse(train_set, 'train')
    mse(dev_set, 'dev')
    mse(test_set, 'test')
#show_summary_statistics()

In [11]:
def preprocess(data):
    values = np.array(data)
    values = values.reshape(-1,1)
    values = values.astype('float32') 
    return values

In [12]:
def plot_losses(model_history, title):
    plt.figure()
    plt.plot(model_history.history['loss'], label='Train')
    plt.plot(model_history.history['val_loss'], label='Dev')
    plt.xlabel('Epochs'); plt.ylabel('Loss (mse)')
    plt.title(title)
    plt.legend(); plt.show()

In [13]:
def plot_predictions(model, X_test, Y_test, Y_prevrawprice, title, inverse=False, scaler=None):
    y_hat = model.predict(X_test)

    if inverse:
        y_hat = inverse_transform(y_hat, Y_prevrawprice, scaler)
        Y_test = inverse_transform(Y_test, Y_prevrawprice, scaler)

    plt.plot(y_hat, label='Predicted')
    plt.plot(Y_test, label='True')
    plt.xlabel('Time'); 

    if inverse:
        plt.ylabel('Price')
    else:
        plt.ylabel('RESCALED Price')

    plt.title(title)
    plt.legend(); plt.show()

In [14]:
def calculate_MSE_RMSE(model, scaler, X_test, Y_test, Y_prevrawprice, model_name):
    y_hat = model.predict(X_test)
    y_hat_inverse = inverse_transform(y_hat, Y_prevrawprice, scaler)
    Y_test_inverse = inverse_transform(Y_test, Y_prevrawprice, scaler)
    mse = mean_squared_error(Y_test_inverse, y_hat_inverse)
    rmse = np.sqrt(mean_squared_error(Y_test_inverse, y_hat_inverse))
    print('%s:' % model_name)
    print('Test MSE: %.3f' % mse)
    print('Test RMSE: %.3f' % rmse)
    print()

In [15]:
def train_evaluate(model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):

    # Train model
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      validation_split=0.05, verbose=verbose, shuffle=False)
    #train_evaluate_showresults(history, model, model_name, 
    #                 X_train, Y_train, X_dev, Y_dev, X_test, Y_test,
    #                 lag, batch_size, epochs, verbose)
    return history

In [16]:
def train_evaluate_showresults(history, model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):
    # Plot losses, predictions, and calculate MSE and RMSE
    plot_losses(history, 'Loss\n(%s)' % model_name)
    plot_predictions(model, X_dev, Y_dev, Y_dev_prevrawprice, 'Test Predictions\n(%s)' % model_name)
    plot_predictions(model, X_dev, Y_dev, Y_dev_prevrawprice, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=price_scaler)
    calculate_MSE_RMSE(model, price_scaler, X_dev, Y_dev, Y_dev_prevrawprice, '%s' % model_name)

In [17]:
def evaluate_test(model, model_name, 
                   X_train, Y_train, Y_train_prevrawprice, X_dev, Y_dev, Y_dev_prevrawprice, X_test, Y_test, Y_test_prevrawprice,
                   lag=10, batch_size=100, epochs=10, verbose=1):
    # Plot losses, predictions, and calculate MSE and RMSE
    #plot_losses(history, 'Loss\n(%s)' % model_name)
    plot_predictions(model, X_test, Y_test, Y_test_prevrawprice, 'Test Predictions\n(%s)' % model_name)
    plot_predictions(model, X_test, Y_test, Y_test_prevrawprice, 'Test Predictions\n(%s)' % model_name, inverse=True, scaler=price_scaler)
    calculate_MSE_RMSE(model, price_scaler, X_test, Y_test, Y_test_prevrawprice, '%s' % model_name)

In [31]:
def initialize_model(X_train, loss, optimizer, num_LSTMs, num_units, dropout, predict_end_of_window):
    
    LSTM_input_shape = [X_train.shape[1], X_train.shape[2]]
    print('input shape is')
    print(LSTM_input_shape)

    # DEFINE MODEL
    model = Sequential()

    if num_LSTMs == 2:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))
            
            if predict_end_of_window:
              model.add(LSTM(num_units[1], return_sequences=False))
            else:
              model.add(LSTM(num_units[1], return_sequences=True))
        
    if num_LSTMs == 3:
            model.add(LSTM(num_units[0], input_shape=LSTM_input_shape, return_sequences=True))
            model.add(Dropout(dropout))

            model.add(LSTM(num_units[1], return_sequences=True))
            model.add(Dropout(dropout))
            
            if predict_end_of_window:
              model.add(LSTM(num_units[2], return_sequences=False))
            else:
              model.add(LSTM(num_units[2], return_sequences=True))

    if predict_end_of_window:
      model.add(TimeDistributed(Dense(1)))
    else:
      model.add(Dense(1))
      
    model.add(Activation('linear'))
    
    
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

In [32]:
def split_X(df):
    n_all = df.shape[0]
    n_train = round(n_all * 0.9)
    n_dev   = round(n_all * 0.05)
    n_test  = round(n_all * 0.05)
    print('n_all:  ', n_all)
    print('n_train:', n_train)
    print('n_dev:  ', n_dev)
    print('n_test: ', n_test)

    X_train = df.iloc[:n_train, 1:16].values.astype('float32')
    X_dev   = df.iloc[n_train:n_train+n_dev, 1:16].values.astype('float32')
    X_test  = df.iloc[n_train+n_dev:, 1:16].values.astype('float32')
    print(X_train.shape)
    print(X_dev.shape)
    print(X_test.shape)

    return X_train, X_dev, X_test

In [33]:
def split_Y(df):
    n_all = df.shape[0]
    n_train = round(n_all * 0.9)
    n_dev   = round(n_all * 0.05)
    n_test  = round(n_all * 0.05)
    Y_train = df.iloc[:n_train, -1:].values.astype('float32')
    Y_dev   = df.iloc[n_train:n_train+n_dev, -1:].values.astype('float32')
    Y_test  = df.iloc[n_train+n_dev:, -1:].values.astype('float32')
    print(Y_train.shape)
    print(Y_dev.shape)
    print(Y_test.shape)
    
    return Y_train, Y_dev, Y_test

In [34]:
def df_to_parquet(df, outfile):
    pq.write_table(pa.Table.from_pandas(df), outfile, compression='snappy')

In [35]:
def direction_prediction(y_true, y_pred):
    prop_correct = np.sum(np.sign(y_pred) == np.sign(y_true)) / (y_true.shape[0])
    return prop_correct

In [37]:
def evaluate_model(model, history, X_train, X_dev, X_test, Y_train, Y_dev, Y_test):
    train_loss = history.history['loss'][-1]
    dev_loss   = history.history['val_loss'][-1]
    test_loss  = model.evaluate(X_test, Y_test, verbose=0)
    
    y_hat_train = model.predict(X_train)
    y_hat_dev   = model.predict(X_dev)
    y_hat_test  = model.predict(X_test)
    
    train_prop_correct = direction_prediction(Y_train, y_hat_train)
    dev_prop_correct   = direction_prediction(Y_dev, y_hat_dev)
    test_prop_correct  = direction_prediction(Y_test, y_hat_test)
    
    evaluation = {'train_loss': train_loss,
                  'dev_loss': dev_loss,
                  'test_loss': test_loss,
                  'train_prop_correct': train_prop_correct,
                  'dev_prop_correct': dev_prop_correct,
                  'test_prop_correct': test_prop_correct}

    return evaluation

In [38]:
def create_sequenced_data(data, window, step):
    sequenced = []
    for minute in range(0, len(data) - window, step):
        chunk = data[minute:minute+window]
        sequenced.append(chunk)
    sequenced = np.array(sequenced)
    return sequenced

In [39]:
def split(df):
    X_train, X_dev, X_test = split_X(df)
    Y_train, Y_dev, Y_test = split_Y(df)
    return X_train, X_dev, X_test, Y_train, Y_dev, Y_test 

In [40]:
def load_data(path, day_start=None, day_end=None):

    # Concatenate dataframes
    files = sorted(glob('%s/*.parquet' % path))
    
    if day_start is not None:
        start = day_start
    else:
        start = 0
    if day_end is not None:
        end = day_start
    else:
        end = len(files)
    files = files[start:end]
    
    all_dataframes = []
    for file in files:
        df = pq.read_table(file).to_pandas()
        all_dataframes.append(df)
    df = pd.concat(all_dataframes)
    return df

In [28]:
def create_all_XY(X_train, X_dev, X_test, Y_train, Y_dev, Y_test, window_size, step, predict_end_of_window):
    X_train = create_sequenced_data(X_train, window=window_size, step=step)
    X_dev   = create_sequenced_data(X_dev,   window=window_size, step=step)
    X_test  = create_sequenced_data(X_test,  window=window_size, step=step)

    if predict_end_of_window:
      Y_train = create_sequenced_data(Y_train, window=window_size, step=step)
      Y_dev   = create_sequenced_data(Y_dev,   window=window_size, step=step)
      Y_test  = create_sequenced_data(Y_test,  window=window_size, step=step)
    else:
      Y_train = create_sequenced_data(Y_train, window=window_size, step=step)
      Y_dev   = create_sequenced_data(Y_dev,   window=window_size, step=step)
      Y_test  = create_sequenced_data(Y_test,  window=window_size, step=step)
    
    print('Train, dev, test shapes:')
    print(Y_train.shape)
    print(X_dev.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_dev.shape)
    print(Y_test.shape)
    
    return X_train, X_dev, X_test, Y_train, Y_dev, Y_test 

In [29]:
def save_model_history(model, history, model_path):
    # serialize model to JSON
    
    if os.path.exists(model_path):
        suffix = ''.join(re.findall(r'\d+', str(datetime.datetime.now())))
        model_path = model_path + '_' + suffix

    os.makedirs(model_path)

    model_json = model.to_json()   
    with open(model_path + '/model.json', 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_path + '/model.h5')

    print("Saved model and history to:\n%s" % model_path)

    with open(model_path + '/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [41]:
def create_all_XY_predEndOfWindow(X_train, X_dev, X_test, Y_train, Y_dev, Y_test, window_size, step):
  
    X_train = create_sequenced_data(X_train, window=window_size, step=step)
    X_dev   = create_sequenced_data(X_dev,   window=window_size, step=step)
    X_test  = create_sequenced_data(X_test,  window=window_size, step=step)

    Y_train = create_sequenced_data(Y_train, window=window_size, step=step)
    Y_dev   = create_sequenced_data(Y_dev,   window=window_size, step=step)
    Y_test  = create_sequenced_data(Y_test,  window=window_size, step=step)
    
    print('Train, dev, test shapes:')
    print(Y_train.shape)
    print(X_dev.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_dev.shape)
    print(Y_test.shape)
    
    return X_train, X_dev, X_test, Y_train, Y_dev, Y_test 


In [42]:
## MAIN MODEL ##

# HYPERPARAMETERS
window_size = 60
step = 1

batch_size = 8192
num_epochs = 50
verbose = 1
loss = 'mean_squared_error'
optimizer = 'adam'
# num_LSTM = 3
# num_units = [128, 256, 256]
num_LSTM = 2
num_units = [256, 256]
dropout = 0.1

path = 'cboe/parquet_preprocessed_BTCUSD_merged'
day_start = 401
day_end = None

model_name = 'window-%s_step-%s_batch-%s_epochs-%s_loss-%s_opt-%s_numLSTMs-%s_numUnits-%s_dropout-%s' % (window_size, step, batch_size, num_epochs, loss, optimizer, num_LSTM, num_units, dropout)
model_path = 'models/%s' % model_name

# LOAD DATA
df = load_data(path, day_start, day_end)

# CREATE XY DATA
X_train, X_dev, X_test, Y_train, Y_dev, Y_test = split(df)

n_all:   658758
n_train: 592882
n_dev:   32938
n_test:  32938
(592882, 15)
(32938, 15)
(32938, 15)
(592882, 1)
(32938, 1)
(32938, 1)


In [49]:
window=3
step=1
print(Y_test[:10])
print(Y_test[window-1:10:window-1])

[[ 0.00068536]
 [-0.00083374]
 [-0.00080732]
 [ 0.00419966]
 [ 0.00125001]
 [-0.00550523]
 [ 0.00177589]
 [ 0.00248398]
 [ 0.        ]
 [-0.00220427]]
[[-0.00080732]
 [ 0.00125001]
 [ 0.00177589]
 [ 0.        ]]


In [ ]:
## MAIN MODEL ##

# HYPERPARAMETERS
window_size = 60
step = 1

batch_size = 8192
num_epochs = 50
verbose = 1
loss = 'mean_squared_error'
optimizer = 'adam'
# num_LSTM = 3
# num_units = [128, 256, 256]
num_LSTM = 2
num_units = [256, 256]
dropout = 0.1

path = 'cboe/parquet_preprocessed_BTCUSD_merged'
day_start = 401
day_end = None

model_name = 'window-%s_step-%s_batch-%s_epochs-%s_loss-%s_opt-%s_numLSTMs-%s_numUnits-%s_dropout-%s' % (window_size, step, batch_size, num_epochs, loss, optimizer, num_LSTM, num_units, dropout)
model_path = 'models/%s' % model_name

# LOAD DATA
df = load_data(path, day_start, day_end)

# CREATE XY DATA
X_train, X_dev, X_test, Y_train, Y_dev, Y_test = split(df)

# X_train, X_dev, X_test, Y_train, Y_dev, Y_test = create_all_XY(X_train, X_dev, X_test, 
#                                                                Y_train, Y_dev, Y_test,
#                                                                window_size, step)

X_train, X_dev, X_test, Y_train, Y_dev, Y_test = create_all_XY_predEndOfWindow(X_train, X_dev, X_test, 
                                                               Y_train, Y_dev, Y_test,
                                                               window_size, step)
# INITIALIZE MODEL
#model = initialize_model(X_train, loss, optimizer, num_LSTM, num_units, dropout, predict_end_of_window=False)
model = initialize_model_predEndOfWindow(X_train, loss, optimizer, num_LSTM, num_units, 
                                         dropout, predict_end_of_window=True)

# TRAIN MODEL
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs,
                      validation_data=(X_dev, Y_dev), verbose=verbose, shuffle=False) 

# SAVE MODEL AND HISTORY
save_model_history(model, history, model_path)

n_all:   658758
n_train: 592882
n_dev:   32938
n_test:  32938
(592882, 15)
(32938, 15)
(32938, 15)
(592882, 1)
(32938, 1)
(32938, 1)
Train, dev, test shapes:
(592822, 60, 1)
(32878, 60, 15)
(32878, 60, 15)
(592822, 60, 1)
(32878, 60, 1)
(32878, 60, 1)
input shape is
[60, 15]
Train on 592822 samples, validate on 32878 samples
Epoch 1/50
592822/592822 [==============================] - 102s 172us/step - loss: 0.0077 - val_loss: 2.2958e-05
Epoch 2/50
592822/592822 [==============================] - 97s 164us/step - loss: 2.4588e-05 - val_loss: 1.3356e-05
Epoch 3/50
592822/592822 [==============================] - 97s 164us/step - loss: 1.4918e-05 - val_loss: 1.0932e-05
Epoch 4/50
592822/592822 [==============================] - 97s 164us/step - loss: 1.2197e-05 - val_loss: 9.4741e-06
Epoch 5/50
592822/592822 [==============================] - 97s 164us/step - loss: 1.0542e-05 - val_loss: 8.2874e-06
Epoch 6/50
592822/592822 [==============================] - 97s 164us/step - loss: 9.3355e-

In [ ]:
# TO RESTORE MODEL:
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
# EVALUATE MODEL
evaluate = evaluate_model(model, history, X_train, X_dev, X_test, Y_train, Y_dev, Y_test)
with open(model_path + '/evaluation.txt', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
def plot_price(df):
    plt.plot(df['current_price'])

In [ ]:
def plot_price(df, X_train, X_dev, field):
    X_train_stop = len(X_train)
    X_dev_stop = X_train_stop + len(X_dev)

    plt.figure(figsize=(20,4))
    plt.plot(np.arange(0, X_train_stop), df.iloc[0:X_train_stop][field], 'k')
    plt.plot(np.arange(X_train_stop, X_dev_stop), df.iloc[X_train_stop:X_dev_stop][field], 'r')
    plt.plot(np.arange(X_dev_stop, len(df)), df.iloc[X_dev_stop:len(df)][field], 'g')

In [ ]:
def plot_train_dev_losses(history):
    train_loss = history.history['loss']
    dev_loss   = history.history['val_loss']
    
    plt.figure(figsize=(20,4))
    plt.plot(np.log(train_loss), 'k')
    
    plt.figure(figsize=(20,4))
    plt.plot(np.log(dev_loss), 'b')
    
    
    plt.figure(figsize=(20,4))
    plt.plot(train_loss, 'k')
    
    plt.figure(figsize=(20,4))
    plt.plot(dev_loss, 'b')
    
    plt.show()

In [ ]:
def plot_percent_change(y_pred, y_true, timestep_within_window, minute_start, minute_end):
    ys=[]
    for i in range(len(y_pred)):
        ys.append(y_pred[i][timestep_within_window])

    original_ys=[]
    for i in range(len(y_true)):
        original_ys.append(y_true[i][timestep_within_window])

    ys_orig = np.array(original_ys)
    ys_pred = np.array(ys)
    
    
    OldRange = (ys_pred.max() - ys_pred.min())  
    NewRange = (ys_orig.max() - ys_orig.min())   
    new_ys_pred = (((ys - ys_pred.min()) * NewRange) / OldRange) + ys_orig.min()
    
    
    norm1 = ys_orig / np.linalg.norm(ys_orig)
    norm2 = ys_pred / np.linalg.norm(ys_pred)
    

    plt.figure(figsize=(20,10))
    plt.plot(norm1[minute_start:minute_end], 'k', alpha=0.9)
    plt.plot(norm2[minute_start:minute_end], 'r', alpha=0.9)
    
    plt.figure(figsize=(20,10))
    plt.plot(original_ys[minute_start:minute_end], 'k', alpha=0.9)
    plt.plot(ys[minute_start:minute_end], 'r', alpha=0.9)
    
    plt.figure(figsize=(20,10))
    plt.plot(original_ys[minute_start:minute_end], 'k', alpha=0.9)
    plt.plot(new_ys_pred[minute_start:minute_end], 'r', alpha=0.9)

In [ ]:
# Get test predictions
y_hat_test = model.predict(X_test)

In [ ]:
# VISUALIZE
## Plot: 
# Historical price, color coded with train, dev, test
# Historical percent change, color coded with train, dev, test
# Train Loss, Dev Loss
# Actual price vs predicted price (or percent change) for test set
# Example features time series for one day (NOTE: in the preprocessing_final notebook)

timestep_within_window = 0
minute_start = 0
minute_end = len(Y_test)

plot_price(df, X_train, X_dev, field='current_price')
plot_price(df, X_train, X_dev, field='percent_change')
plot_train_dev_losses(history)
plot_percent_change(y_hat_test, Y_test, timestep_within_window, minute_start, minute_end)